# 合成数据
基于上述各种例子，数据的生成往往包含格式的约束，我们在构建数据集的时候当然更希望生成标准格式的数据，那样我们在数据清洗和整理的时候能更加轻松，大模型包含广泛的知识数据，生成数据的时候并不会偏向某一方，比如医疗、金融等特定领域任务，而且由于有良好的上下文理解能力，给够足够的指令约束，数据生成的效果会比预想中的更加准确

> 这里仅简单举例，下面我们分了单独的章节详细讲解预训练、微调、推理等合成数据如何生成。

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.17it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [3]:
prompt="""请按下列要求，按规定格式生成单轮问答的数据：
询问并解释什么是LLM
格式: {"input":"...","output":"..."}"""

In [4]:
print(inference(model_instruct,tokenizer_instruct,prompt))


{'response': '{"input":"什么是LLM？","output":"LLM代表Large Language Model，即大型语言模型。这是一种深度学习模型，能够理解和生成人类语言，常用于自然语言处理任务中，如文本生成、对话系统等。"}', 'generated_tokens_count': 51}
